# Calculate daily average potential evaporation from 3-year dataset for Source Model

*Written by Nop Channpisey & Men Vuthy, 2022*

---



### Objective



*   The objective is to calculate daily average potential evaporation from 3-year dataset.
*   Save the result data as ASCII file for Source Model



3 years of daily PET Data:

![img](img-markdown/pet-data-1.png "Title")

### Code

*Installation*

In [ ]:
!pip install geopandas
!pip install rasterio

In [2]:
# Import modules
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio

import matplotlib.pyplot as plt
%matplotlib inline

from collections import defaultdict
from pyproj import CRS

In [3]:
cd /content/drive/MyDrive/Dataset/Mekong Water Solution

/content/drive/MyDrive/Dataset/Mekong Water Solution


*Create path of all image files in folder*

In [4]:
# File and folder paths
file_path = "PET_ASC_2014-2016"

# Make a search criteria to select the ndvi files
query = os.path.join(file_path, "20*.asc")

pet_fp = sorted(glob.glob(query)) # sorted files by name

print(pet_fp)

['PET_ASC_2014-2016/20140101_pet.asc', 'PET_ASC_2014-2016/20140102_pet.asc', 'PET_ASC_2014-2016/20140103_pet.asc', 'PET_ASC_2014-2016/20140104_pet.asc', 'PET_ASC_2014-2016/20140105_pet.asc', 'PET_ASC_2014-2016/20140106_pet.asc', 'PET_ASC_2014-2016/20140107_pet.asc', 'PET_ASC_2014-2016/20140108_pet.asc', 'PET_ASC_2014-2016/20140109_pet.asc', 'PET_ASC_2014-2016/20140110_pet.asc', 'PET_ASC_2014-2016/20140111_pet.asc', 'PET_ASC_2014-2016/20140112_pet.asc', 'PET_ASC_2014-2016/20140113_pet.asc', 'PET_ASC_2014-2016/20140114_pet.asc', 'PET_ASC_2014-2016/20140115_pet.asc', 'PET_ASC_2014-2016/20140116_pet.asc', 'PET_ASC_2014-2016/20140117_pet.asc', 'PET_ASC_2014-2016/20140118_pet.asc', 'PET_ASC_2014-2016/20140119_pet.asc', 'PET_ASC_2014-2016/20140120_pet.asc', 'PET_ASC_2014-2016/20140121_pet.asc', 'PET_ASC_2014-2016/20140122_pet.asc', 'PET_ASC_2014-2016/20140123_pet.asc', 'PET_ASC_2014-2016/20140124_pet.asc', 'PET_ASC_2014-2016/20140125_pet.asc', 'PET_ASC_2014-2016/20140126_pet.asc', 'PET_ASC_20

*Read all image files in folder*

In [5]:
# Create a list to store image file
pet_file = []

# Read each file one by one and store into above list
for path in pet_fp:

    # Read image using rasterio
    image = rasterio.open(path)

    # Read first band/channel
    image = image.read(1)

    # Write/Append image to storage list
    pet_file.append(image)

*Find the index of the same date of each year*

In [6]:
# Create a list of file name extension
nameExt = []

# Extract file name extension one by one
for fileName in range(len(pet_fp)):
  # Select only date
  extension = str(pet_fp[fileName][-12:])
  # Append to above list
  nameExt.append(extension)

# Collect the index of same date of each year
# Create a collections dictionary
indexCollection = defaultdict(list)

# Find the index of the same date of each year
for i, x in enumerate(nameExt):
  # Append index of the same date to collection dictionary above
  indexCollection[x].append(i)

# Create a list to store index value
indexList = list(indexCollection.values())

*Calculate mean array from first to last date of the year*

In [7]:
# List to store mean array of each date
meanList = []

# Loop to calculate mean array one date by one date
for date, file in enumerate(indexList):

  # Create a list to store mean array of one day
  arrayByDate = []

  # Loop to calculate mean array of one day
  for i in file:

    # Append array on the same date of all years to arrayByDate list
    arrayByDate.append(pet_file[i])

  # Calculate mean array on the same date of all years
  meanByDay = np.mean(arrayByDate, axis=0)

  # Append to mean array from first to last date of the year to meanList
  meanList.append(meanByDay)


*Export output as ASCII file to drive*

In [8]:
# Read one image as reference format
img_ref = rasterio.open('PET_ASC_2014-2016/20140101_pet.asc')

# Define coordinate reference system to WGS 84 - EPSG:4326
crs_wgs84 = CRS.from_user_input(4326)

# Export every output as ASCII file
for i in range(len(meanList)-1):
  # File name
  name = str('2017'+pet_fp[i][-12:])

  # Data dir
  data_dir = 'Output/PET_ASC_Mean_2017'

  # Output ascii
  out_ascii = os.path.join(data_dir, name)

  # Copy the metadata
  out_meta = img_ref.meta.copy()

  # update meta
  out_meta.update({'driver': 'AAIGrid',
                 'dtype': 'float32',
                 'nodata': -9999.0,
                 'width': meanList[i].shape[1],
                 'height': meanList[i].shape[0],
                 'crs': crs_wgs84,
                 'count': 1,
                 'transform': img_ref.transform,
                })
  
  # Write image to ASCII file
  with rasterio.open(out_ascii, "w", **out_meta) as dest:
        dest.write(meanList[i].astype(np.float32), indexes=1)

Daily averageg PET output:

![img2](img-markdown/pet-avg-2.png "Title")

![img3](img-markdown/pet-avg-1.png "Title")